# Make fake flat, LFC, and science exposure

In [ ]:
import numpy as np
import pylab as plt
%matplotlib inline

In [ ]:
# Set defaults.
specvar = 6. # pixels ** 2
rng = np.random.default_rng(17)
TWOPI = 2. * np.pi
ONEOVERTWOPI = 1. / TWOPI

In [ ]:
# Define useful functions.
def gaussian_2d(xxs, yys, mean, var):
    """
    `xxs`   should be shape `(nx, ny)`.
    `yys`   should be shape `(nx, ny)`.
    `mean`  should be shape `(2, )`.
    `var`   should be a scalar.
    """
    return ONEOVERTWOPI / var \
           * np.exp(-0.5 / var * ((xxs - mean[0]) ** 2 + (yys - mean[1]) ** 2))
def gaussian_1d(xxs, mean, var):
    """
    `xxs`   should be shape `(nx, )`.
    `mean`  should be a scalar.
    `var`   should be a scalar.
    """
    return np.sqrt(ONEOVERTWOPI / var) \
           * np.exp(-0.5 / var * (xxs - mean) ** 2)

In [ ]:
# Make true 1D spectrum.
linexs = 2. + np.array([6.1, 20.2, 37.0])
depths = 5. * np.array([0.2, 0.7, 0.1])

In [ ]:
xfine = np.arange(0., 45., 0.01)
yfine = np.ones_like(xfine)
for x, d in zip(linexs, depths):
    yfine -= d * gaussian_1d(xfine, x, specvar)

In [ ]:
plt.title("true 1D spectrum")
plt.plot(xfine, yfine, 'k-')

In [ ]:
# Define trace and the non-orthogonality of the wavelength solution.
def trace(xs):
    return 12.83 - 0.003 * (xs - 50.3) ** 2
def dxdy(xs):
    return - 0.006 * (xs - 50.3)

In [ ]:
# Make pixel grids for a tiny image section
xs = np.arange(43).astype(float)
ys = np.arange(21).astype(float)
xxs, yys = np.meshgrid(xs, ys)

In [ ]:
plt.title("just checking")
plt.plot(xs, trace(xs), "k-")
plt.axis("equal")
dy = 5.
for x in np.arange(0, len(xs), 2).astype(float):
    y0 = trace(x)
    plt.plot([x - dxdy(x) * dy, x + dxdy(x) * dy], [y0 - dy, y0 + dy], "k-")

In [ ]:
# Make true flat by an agonizing process.
xspots = np.arange(-4., len(xs) + 4., 0.33)
yspots = np.arange(-5., 5., 0.33)
var = 1.0
trueflat = np.zeros_like(xxs).astype(float)
for x in xspots:
    for y in yspots:
        trueflat += gaussian_2d(xxs, yys, [x + dxdy(x) * y, trace(x) + y], var)

In [ ]:
# Make true LFC by a similarly agonizing process.
lfcspots = np.arange(-4., len(xs) + 4., 4 * np.pi)
truelfc = np.zeros_like(xxs).astype(float)
for x in lfcspots:
    for y in yspots:
        truelfc += gaussian_2d(xxs, yys, [x + dxdy(x) * y, trace(x) + y], specvar)

In [ ]:
# Make true science image by multiplying the flat.
foo = np.ones_like(xxs)
for x, d in zip(linexs, depths):
    foo -= d * gaussian_1d(xxs - dxdy(xxs) * yys, x, specvar)
truesci = trueflat * foo

In [ ]:
# Make observed images by noisifying the true images.
flat = trueflat + 0.05 * np.sqrt(trueflat) * rng.normal(size=xxs.shape)
lfc  = truelfc  + 0.01 * np.sqrt(truelfc)  * rng.normal(size=xxs.shape)
sci  = truesci  + 0.05 * np.sqrt(truesci)  * rng.normal(size=xxs.shape)

In [ ]:
imshowkwargs = {"origin": "lower", "interpolation": "nearest"}
plt.title("flat")
plt.imshow(flat, **imshowkwargs)
plt.plot(lfcspots, trace(lfcspots), "ro")
plt.xlim(-0.5, len(xs)-0.5)

In [ ]:
plt.title("LFC")
plt.imshow(lfc, **imshowkwargs)
plt.plot(lfcspots, trace(lfcspots), "ro")
plt.xlim(-0.5, len(xs)-0.5)

In [ ]:
plt.title("science")
plt.imshow(sci, **imshowkwargs)
plt.plot(lfcspots, trace(lfcspots), "ro")
plt.xlim(-0.5, len(xs)-0.5)